In [93]:
import pandas as pd
import pprint
import ast
import json
import numpy as np
from scipy.stats import entropy
from scipy.special import softmax
import numpy as np
import pandas as pd
from scipy.special import softmax
from scipy.stats import entropy
import ast



In [94]:
from dict import newsgroups_labels

In [95]:
dict_map = newsgroups_labels

In [96]:

def calculate_metric_with_true_label(df, column_name):
    

    def parse_top_logprobs(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"❌ Nie udało się sparsować: {val[:60]}... ➡ {e}")
                return None
        return val

    df['top_logprobs'] = df['top_logprobs'].apply(parse_top_logprobs)
    
    df['logprobs_cal'] = df['top_logprobs'].apply(
    lambda x: np.array([item['logprob'] for item in x if isinstance(item, dict) and 'logprob' in item])
    if isinstance(x, list) else np.array([])
)



    # Przekształcenie logprobs do prawdopodobieństw przez softmax
    df['probs'] = df['logprobs_cal'].apply(
        lambda logprobs: softmax(logprobs) if len(logprobs) > 0 else np.array([])
    )

    # Entropia rozkładu
    df['entropy'] = df['probs'].apply(
        lambda probs: entropy(probs, base=2) if len(probs) > 0 else np.nan
    )

    # Margin: różnica między top-1 a top-2
    df['margin'] = df['probs'].apply(
        lambda probs: np.sort(probs)[-1] - np.sort(probs)[-2] if len(probs) > 1 else np.nan
    )

    # KL divergence względem idealnego rozkładu (1.0 na poprawnej etykiecie)
    def kl_div(probs, top_logprobs, label):
        if not isinstance(top_logprobs, list) or len(probs) == 0:
            return np.nan

        try:
            # lista tokenów jako stringi
            tokens = [str(item['token']).strip().lower() for item in top_logprobs if isinstance(item, dict) and 'token' in item]
            label_str = str(label).strip().lower()

            if label_str not in tokens:
                return np.nan

            match_idx = tokens.index(label_str)
            ideal_dist = np.zeros_like(probs)
            ideal_dist[match_idx] = 1.0

            return entropy(ideal_dist, probs, base=2)

        except Exception as e:
            print(f"⚠️ Błąd w kl_div: {e}")
            return np.nan


    df['kl_divergence'] = df.apply(
        lambda row: kl_div(row['probs'], row['top_logprobs'], row[column_name]), axis=1
    )

    # Normalizacja (min-max)
    for col in ['entropy', 'margin', 'kl_divergence']:
        norm_col = f'normalized_{col}'
        min_val = df[col].min()
        max_val = df[col].max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val > min_val:
            df[norm_col] = (df[col] - min_val) / (max_val - min_val)
        else:
            df[norm_col] = 0.0

    return df




In [97]:
model = 'gpt4omini'
dataset_tag = '20_newsgroups'
prompt = 'no_samples_numbers'
temp = 0
exp = 1

In [98]:

path = f'../NO_EXAMPLES/results/{prompt}_temp{0}/{model}/{dataset_tag}_{model}_{prompt}_temp{temp}_exp{exp}.csv'
df = pd.read_csv(path).sample(n=150, random_state=43)
      


In [99]:
df

,text,output,logprobs,top_logprobs,original_label
1480,looking package takes inputs set geometric obj...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.1...",comp.graphics
1312,article mark singer writes exuberance disappea...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -1.8...",rec.sport.baseball
305,jim halat writes article ken arromdee writes a...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",talk.religion.misc
1545,article pack rat writes something bothering wa...,7,"{'content': [{'token': '7', 'bytes': [55], 'lo...","[{'token': '7', 'bytes': [55], 'logprob': -2.5...",sci.space
1716,article pat writes article henry spencer write...,7,"{'content': [{'token': '7', 'bytes': [55], 'lo...","[{'token': '7', 'bytes': [55], 'logprob': -0.0...",sci.space
...,...,...,...,...,...
888,simple eh rather get em used new must guarantee,13,"{'content': [{'token': '13', 'bytes': [49, 51]...","[{'token': '13', 'bytes': [49, 51], 'logprob':...",misc.forsale
1439,anyone know causes evergrowing black border ar...,8,"{'content': [{'token': '8', 'bytes': [56], 'lo...","[{'token': '8', 'bytes': [56], 'logprob': -3.3...",sci.electronics
1053,condition propane tank mentioned bd survivors ...,18,"{'content': [{'token': '18', 'bytes': [49, 56]...","[{'token': '18', 'bytes': [49, 56], 'logprob':...",talk.politics.guns
152,sale voice processing system ibm compatibles i...,13,"{'content': [{'token': '13', 'bytes': [49, 51]...","[{'token': '13', 'bytes': [49, 51], 'logprob':...",misc.forsale


In [100]:
df['original_label_map'] = df['original_label'].map(dict_map)

In [101]:
df['is_error'] = df['original_label_map'] != df['output']

In [102]:
df_metrics = calculate_metric_with_true_label(df, 'original_label_map')


In [103]:
corr_matrix = df_metrics[['is_error', 'entropy', 'margin', 'kl_divergence']].corr(method='pearson')
print(corr_matrix['is_error'])


is_error         1.000000
entropy          0.391039
margin          -0.358874
kl_divergence    0.864034
Name: is_error, dtype: float64


In [104]:
from scipy.stats import pointbiserialr

r, p = pointbiserialr(df_metrics['is_error'], df_metrics['entropy'])
print(f"Point-biserial r = {r:.4f}, p = {p:.4e}")


Point-biserial r = 0.3910, p = 7.5127e-07


In [105]:
r, p = pointbiserialr(df_metrics['is_error'], df_metrics['margin'])
print(f"Point-biserial r = {r:.4f}, p = {p:.4e}")

Point-biserial r = -0.3589, p = 6.4929e-06


In [106]:
df_nan_kl = df[df['kl_divergence'].isna()]
df_nan_kl


,text,output,logprobs,top_logprobs,original_label,original_label_map,is_error,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence


In [107]:
df_filtered = df.dropna(subset=['is_error', 'kl_divergence'])

r, p = pointbiserialr(df_filtered['is_error'], df_filtered['kl_divergence'])
print(f"r = {r:.4f}, p = {p:.4e}")

r = 0.8640, p = 5.8340e-46
